## Imports

In [1]:
import os
import json 

## Settings

In [2]:

setting_file = os.path.abspath(os.path.join('..', 'settings.json'))

with open(setting_file, 'r') as f:
    settings = json.load(f)


## Connect to and prepare database
Use SqlAlchemy to autoload tables

In [3]:
# use sqlalchemy and reflection to get the table names
from sqlalchemy import create_engine, MetaData, Table, func, Column, Integer, String, Text, ForeignKey, UniqueConstraint, exc
from sqlalchemy.orm import sessionmaker

# create the engine
engine = create_engine(settings['sqlalchemy_database_uri'])

# Reflect the tables
metadata = MetaData()
metadata.reflect(bind=engine)

# create connection
conn = engine.connect()
Session = sessionmaker(bind=engine)
session = Session()


"""
Creates the database if it does not exist
"""
# open database connection using sqlalchemy
engine = create_engine(settings['sqlalchemy_database_uri'])
# Create the Metadata Object 
metadata_obj = MetaData() 

# Define the profile table    
# database name 
Prototypes = Table( 
    'prototypes',                                         
    metadata_obj,                                     
    Column('hash', String, primary_key=True),   
    Column('name', String),
    Column('prompt', Text),
    Column('language', String),
    Column('code', Text),
    Column('xml', Text),
    Column('status', Integer),
    Column('num_errors', Integer),               
) 

# Define the ingredient table
Ingredient = Table( 
    'ingredient', 
    metadata_obj, 
    Column('id', Integer, primary_key = True, autoincrement=True), 
    Column('prototype', String, ForeignKey('prototypes.hash'), nullable=False),
    Column('tag', String), 
    Column('position', Integer),
    Column('depth', Integer),
    # create unique constraint
    UniqueConstraint('prototype', 'position', name='unique_ingredient') 
)

# Create the Table 
metadata_obj.create_all(engine) 

# reflect the tables
metadata_obj.reflect(bind=engine)

# check if the prototypes table contains a column named xml (older versions of the database do not have this column)
contains_xml = False
# print column names
for column in Prototypes.columns:
    if column.name == 'xml':
        contains_xml = True

if not contains_xml: 
    raise Exception('The prototypes table does not contain a column named xml')


# load other tables
Analysis = Table('analysis', metadata, autoload_with=engine)
Sample = Table('sample', metadata, autoload_with=engine)
Tag = Table('tag', metadata, autoload_with=engine)
SampleTag = Table('sample_tag', metadata, autoload_with=engine)

## Retrieve and write prototype code to file

In [4]:
# get one prototype
prototype = session.query(Prototypes).first()

tmp_dir = os.path.abspath('tmp')
os.makedirs(tmp_dir, exist_ok=True)

# write the prototype to a file
prototype_source = os.path.join(tmp_dir, 'prototype.c')
with open(prototype_source, 'w') as f:
    f.write(prototype.code)


## Create XML file of prototype code

In [5]:
srcml_client = settings['srcml_client']

# use subprocess to run the srcml client
import subprocess


# compile the code
prototype_xml = os.path.join(tmp_dir, 'prototype.xml')            
result = subprocess.run([srcml_client, prototype_source, '-o', prototype_xml], capture_output=True, text=True)

# check if the command was successful
if result.returncode != 0:
    print(result.stderr)
    raise Exception('Failed to run srcml client')
else:
    print(result.stdout)


## Read XML file into python object

In [6]:
# read file at location prototype_xml into xml object

import xml.etree.ElementTree as ET

tree = ET.parse(prototype_xml)
root = tree.getroot()

# # Traverse all elements in the XML document (example)
# for elem in root.iter():
#     print(elem.tag.split('}')[1], elem.attrib, elem.text)

In [7]:
# example of using namespaces in findall
ns = {'src': 'http://www.srcML.org/srcML/src',
      'cpp': 'http://www.srcML.org/srcML/cpp'}
for child in root.findall('src:comment', ns):
    print(child.text)

// Structure for queue node
// Structure for queue
// Function to create a new queue node
// Function to create an empty queue
// Function to check if a queue is empty
// Function to enqueue an item
// Function to dequeue an item
// Function to perform breadth-first search on a directory tree
// Main function


## Function for finding the depth of a node 
(actually, reverse depth, aka the deepest number of children)

In [8]:
def find_depth(elem, depth=0):
    """Find the depth of an element in the XML tree."""
    # Base case: if the element has no children, return the current depth
    if len(elem) == 0:
        return depth
    # Recursive case: return the maximum depth of the element's children
    else:
        return max(find_depth(child, depth + 1) for child in elem)

## Add ingredients to database

In [9]:
position = 0
for elem in root.iter():
    depth = find_depth(elem)
    tag = elem.tag.split('}')[1]
    print(f"Adding {tag} at position {position} with depth {depth} to the database for prototype {prototype.hash}.")
    query = Ingredient.insert().values(prototype=prototype.hash, tag=tag, position=position, depth=depth)
    try:
        conn.execute(query)
    except exc.IntegrityError as e:
        print(f"Failed to add {tag} at position {position} with depth {depth} to the database for prototype {prototype.hash}.")
        conn.rollback()
    position += 1
conn.commit()

Adding unit at position 0 with depth 38 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding include at position 1 with depth 1 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding directive at position 2 with depth 0 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding file at position 3 with depth 0 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding include at position 4 with depth 1 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding directive at position 5 with depth 0 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding file at position 6 with depth 0 to the database for prototype 8795114f5020f373079e1e573b06265db58a7c2f6f219c5bb5a6c600b5eb0113.
Adding include at position 7 with dept

In [10]:
# close the connection
conn.close()
session.close()
engine.dispose()
print('Done')

Done
